以下の参考サイトの手順をTensorflow1で実行するように変更した。  
参考：  https://towardsdatascience.com/train-an-object-detector-using-tensorflow-2-object-detection-api-in-2021-a4fed450d1b9

# 事前準備
ランタイム→ランタイムのタイプを変更を選択、ハードウェアアクセラレータを「GPU」に変更しておく。


# GoogleDriveのマウント
GoogleDrive上にファイルを保存する場合は以下のセルを実行してカレントディレクトリを移動しておく(CTRL+M Yでコードセル化)。    
ただし、3GByte程度消費するので空き容量に注意。  
不要な場合はスキップする(CTRL+M Mでテキストセル化)。    


from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

# Tensorflowのバージョンを1.xに変更する。

In [ ]:
%tensorflow_version 1.x

【NOTE】  
以下でTensorflowのバージョンを変更しようとすると、なぜかCPU版になってしまう。
``tensorflow-gpu``だとtensorflowが見つからないと怒られる。  
```
!pip uninstall -y tensorflow
!pip install --upgrade tensorflow-gpu==1.15.5
```

# ワークディレクトリの作成と移動

In [ ]:
import sys
import os

!mkdir -p hand_detect_tf1
%cd hand_detect_tf1

# 現在のディレクトリ
CUR_DIR = os.getcwd()

# object-detection モジュールのインストール
## gitリポジトリのclone

In [ ]:
%cd $CUR_DIR
!git clone --depth 1 https://github.com/tensorflow/models.git

## プロトコルバッファのコンパイル

In [ ]:
%cd $CUR_DIR/models/research
!protoc object_detection/protos/*.proto --python_out=.

## モジュールのインストール

In [ ]:
%cd $CUR_DIR/models/research
!cp object_detection/packages/tf1/setup.py .
!python -m pip install .

## テスト

In [ ]:
%cd $CUR_DIR/models/research
!python object_detection/builders/model_builder_tf1_test.py

# データセットのダウンロード
## gitリポジトリのclone

In [ ]:
%cd $CUR_DIR
!git clone https://github.com/aalpatya/detect_hands.git

##  データのダウンロード→csvファイルの生成→tf_recordsの生成

In [ ]:
%cd $CUR_DIR
!python detect_hands/egohands_dataset_to_csv.py
!python detect_hands/generate_tfrecord.py --csv_input=images/train/train_labels.csv  --output_path=train.record
!python detect_hands/generate_tfrecord.py --csv_input=images/test/test_labels.csv  --output_path=test.record

# 元となるモデルのダウンロード

元になるモデルファイルは以下を参照  
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md


In [ ]:
%cd $CUR_DIR
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar xzvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz 

## pipline.txtの修正
学習処理に合わせて修正する

In [ ]:
%cd $CUR_DIR

# 学習回数
NUM_STEPS = 80000                  # 回数は要検討

# 学習済みモデル出力ディレクトリ
TRAINED_DIR = "output_training"
TRAINED_DIR_ABS = f"{CUR_DIR}/{TRAINED_DIR}"
!mkdir -p $TRAINED_DIR_ABS

ORG_MODEL_NAME = "ssd_mobilenet_v2_coco_2018_03_29"

# オリジナルCONFIGファイル名
ORG_CONFIG_FILE = f"{CUR_DIR}/{ORG_MODEL_NAME}/pipeline.config"
# オリジナルラベルファイル
ORG_LABEL_FILE = f"{CUR_DIR}/detect_hands/model_data/ssd_mobilenet_v2_fpn_320/label_map.pbtxt"

# CONFIGファイル名
CONFIG_FILE = TRAINED_DIR_ABS + "/pipeline.config"
# ラベルファイル
LABEL_FILE = TRAINED_DIR_ABS + "/label_map.pbtxt"

# それぞれをコピー
!cp $ORG_CONFIG_FILE $CONFIG_FILE
!cp $ORG_LABEL_FILE $LABEL_FILE

# オリジナルCHECKPOINTファイル(学習再開する場合はここを変更)
FINE_TUNE_CKPT   = f"{CUR_DIR}/{ORG_MODEL_NAME}/model.ckpt"
# 学習データ
TRAIN_RECORD = f"{CUR_DIR}/train.record"
# テストデータ
TEST_RECORD  = f"{CUR_DIR}/test.record"

import re
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import tensorflow.compat.v1 as tf

# オリジナルファイル読み込み
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(CONFIG_FILE, "r") as f:
  proto_str = f.read()
  proto_str = re.sub('batch_norm_trainable:.*\\n', '',  proto_str)      # batch_norm_trainable エントリを削除(エラー回避)
  text_format.Merge(proto_str, pipeline_config)

# パラメータ変更
pipeline_config.model.ssd.num_classes                                      = 1

pipeline_config.train_config.batch_size                                    = 4
pipeline_config.train_config.fine_tune_checkpoint                          = FINE_TUNE_CKPT
pipeline_config.train_config.num_steps                                     = NUM_STEPS
pipeline_config.train_config.fine_tune_checkpoint_type                     = "detection"

pipeline_config.train_input_reader.label_map_path                          = LABEL_FILE
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0]    = TRAIN_RECORD

pipeline_config.eval_input_reader[0].label_map_path                        = LABEL_FILE
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0]  = TEST_RECORD

# 変更後データの書き込み
pipeline_text = text_format.MessageToString(pipeline_config)
with tf.gfile.Open(CONFIG_FILE, "wb") as f:
  f.write(pipeline_text)
print("DONE")


# TensorBoardの起動
ローカルだとこれでちゃんと結果見れるのに、Google Colab だと見れない...なんで？  

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$TRAINED_DIR_ABS

# 学習の実行
メインイベント  
ちょっと時間がかかるのでお茶でも飲んでてください(数時間のオーダー)

In [ ]:
%cd $CUR_DIR/models/research/object_detection/

!python model_main.py \
--pipeline_config_path=$CONFIG_FILE \
--model_dir=$TRAINED_DIR_ABS \
--alsologtostderr

# モデルのエクスポート
生成したZIPファイルをダウンロードしてください。  

In [ ]:
%cd $CUR_DIR/models/research/object_detection
EXPORT_DIR = "inference"
EXPORT_DIR_ABS = f"{CUR_DIR}/{EXPORT_DIR}"

CHKPT_PREFIX = f"{TRAINED_DIR_ABS}/model.ckpt-{NUM_STEPS}"

!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path $CONFIG_FILE \
--trained_checkpoint_prefix $CHKPT_PREFIX \
--output_directory $EXPORT_DIR_ABS

# ラベルファイルもコピーしておく
!cp $LABEL_FILE $EXPORT_DIR_ABS

In [ ]:
%cd $CUR_DIR

import datetime
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST')
date_str = datetime.datetime.now(JST).strftime('_%Y%m%d_%H%M%S.zip')
zip_filename_export = EXPORT_DIR + date_str
zip_filename_trained = TRAINED_DIR + date_str

zip_filename = datetime.datetime.now().strftime('hand_detect_%Y%m%d_%H%M%S.zip')
!zip -r $zip_filename_export $EXPORT_DIR
!zip -r $zip_filename_trained $TRAINED_DIR

# テスト

In [ ]:
# テスト用画像ファイルのダウンロード
!wget https://cdn.amebaowndme.com/madrid-prd/madrid-web/images/sites/483796/1357355de6edbc4c4b54d22faf0b0756_ce052e9b134a9dbb047a8e17c890832a.jpg -O a.jpg
!wget https://cdn.amebaowndme.com/madrid-prd/madrid-web/images/sites/483796/564b6ca69e9022aa1977f335a148a05a_2d642c807aaf8f5b972a0a406903447d.jpg -O b.jpg


In [ ]:
import os
import sys
import cv2

import numpy as np
import tensorflow as tf

from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# ラベルマップのロード
PATH_TO_LABELS = CUR_DIR + '/inference/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# テスト用イメージファイル
TEST_IMAGE_PATHS = [
                        "a.jpg", 
                        "b.jpg",
                    ]

# モデルのロード
PATH_TO_FROZEN_GRAPH = CUR_DIR + "/inference/frozen_inference_graph.pb"
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict
  




def show_inference(image_path):
  # 画像の読み込み
  image = Image.open(image_path)
  image_np = load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)

  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  # plt.figure(figsize=IMAGE_SIZE)
  # plt.imshow(image_np)

  # 表示
  display(Image.fromarray(image_np))
  # ～～～ 単独実行するときの表示処理はこちら ～～～
  # new_image = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
  # cv2.imshow("Detection Results", new_image)  
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()

# 実行
for image_path in TEST_IMAGE_PATHS:
  show_inference(image_path)
  